In [2]:
import numpy as np
import pandas as pd
import tensorflow.keras as K
import tensorflow as tf
import psycopg2
from decouple import config
import urllib.parse as up
from nltk.stem import WordNetLemmatizer

In [3]:
url = up.urlparse(config("DATABASE_URL"))

In [4]:
conn2 = psycopg2.connect(database=url.path[1:],
                         user=url.username,
                         password=url.password,
                         host=url.hostname,
                         port=url.port)

In [5]:
#Simple function to make sure my SQL DB is connected.
def postgres_test():
    try:
        conn2 = psycopg2.connect(database=url.path[1:],
                         user=url.username,
                         password=url.password,
                         host=url.hostname,
                         port=url.port)
        conn2.close()
        return True
    except:
        return False

In [6]:
postgres_test()

True

In [7]:
#Testing individual user queries, but given mvp goals this may not be ideal
curs2 = conn2.cursor()
#Switched to pulling all data into pandas DF
df = pd.read_sql('SELECT * FROM "HNTroll"', conn2)


In [8]:
df.head()

,id,by,author,time_ts,text,parent,ranking,score
0,2701393,5l,5l,2011-06-27 14:28:01+00:00,And the glazier who fixed all the broken windo...,2701243,0,-0.0616
1,5811403,99,99,2013-06-03 04:34:08+00:00,Does canada have the equivalent of H1B/Green c...,5804452,0,0.0000
2,21623,AF,AF,2007-05-12 17:53:20+00:00,"Speaking of Rails, there are other options in ...",21611,0,0.5205
3,10159727,EA,EA,2015-09-02 15:09:34+00:00,Humans and large livestock (and maybe even pet...,10159396,0,1.0102
4,2988424,Iv,Iv,2011-09-12 18:53:00+00:00,I must say I reacted in the same way when I re...,2988179,0,0.0000


In [9]:
df2 = df.copy()
df2["text"] = df2["text"].str.replace('[^\w\s]','')
df2["text"] = df2["text"].str.replace('^https?:\/\/.*[\r\n]*', '')
df2["text"] = df2["text"].str.replace('x27', '')
df2["text"] = df2["text"].str.replace('</?[A-Za-z]+>', '')

In [10]:
df2["text"] = df2["text"].str.lower()

In [11]:
df2.sample(5)

,id,by,author,time_ts,text,parent,ranking,score
28588,2754627,rglover,rglover,2011-07-12 13:30:55+00:00,how about using the tumblr2wordpress tool by w...,2754490,0,0.9735
16241,6791678,fat0wl,fat0wl,2013-11-24 23:30:22+00:00,hmmm thats true the issue is im a quotfrontend...,6791455,0,0.8504
11759,1887611,secos,secos,2010-11-09 19:31:49+00:00,are you meaning two people on a project if so...,1887315,0,-0.4650
28041,3052967,pnathan,pnathan,2011-09-29 16:00:51+00:00,my local staples sells kindles i would suggest...,3051440,0,0.0000
14362,7072355,asdasf,asdasf,2014-01-16 20:36:35+00:00,two of them quotcombining effortsquot is remov...,7072073,0,-0.1979


In [12]:
lemmatizer = WordNetLemmatizer()

In [13]:
for i in df2['text']:
    i = lemmatizer.lemmatize(i)
    print(i)

e favorite systems programming interview questionsp tell me about some routines you tend to take with you from project to project whats in libyouso crap devs wont have anyp your code crashes inside the allocator walk me through debugging itp tell me about the most interesting problem you debugged without relying on the debugger or print statementspa question i always ask in dev interviews that isnt as systemsy is your typical linked list code does it embed the list pointer in the struct or do you have a separate list node struct with a void in itpi like the questions that have no direct right answerspone question i think you should always ask in every dev interview i am going to describe project xxx its relatively small give me a complete detailed estimate of how long it will take you to do xxx i like this question because 1 it flags people who cant estimate properly 2 it forces them to talk through an actual project and break down the steps 3 it catches people who dont think about tes

In [14]:
df2.head()

,id,by,author,time_ts,text,parent,ranking,score
0,2701393,5l,5l,2011-06-27 14:28:01+00:00,and the glazier who fixed all the broken windo...,2701243,0,-0.0616
1,5811403,99,99,2013-06-03 04:34:08+00:00,does canada have the equivalent of h1bgreen ca...,5804452,0,0.0000
2,21623,AF,AF,2007-05-12 17:53:20+00:00,speaking of rails there are other options in t...,21611,0,0.5205
3,10159727,EA,EA,2015-09-02 15:09:34+00:00,humans and large livestock and maybe even pets...,10159396,0,1.0102
4,2988424,Iv,Iv,2011-09-12 18:53:00+00:00,i must say i reacted in the same way when i re...,2988179,0,0.0000


In [15]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
#sid.polarity_scores(sentence)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\charl\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [16]:
scores = []
for i in df2["text"]:
  score = sid.polarity_scores(i)
  scores.append(score)
scores

8, 'neu': 0.738, 'pos': 0.174, 'compound': 0.5994},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.02, 'neu': 0.88, 'pos': 0.1, 'compound': 0.7304},
 {'neg': 0.052, 'neu': 0.753, 'pos': 0.194, 'compound': 0.8047},
 {'neg': 0.016, 'neu': 0.858, 'pos': 0.125, 'compound': 0.8242},
 {'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound': 0.0},
 {'neg': 0.012, 'neu': 0.916, 'pos': 0.072, 'compound': 0.6827},
 {'neg': 0.032, 'neu': 0.93, 'pos': 0.038, 'compound': 0.1002},
 {'neg': 0.102, 'neu': 0.834, 'pos': 0.064, 'compound': -0.7409},
 {'neg': 0.0, 'neu': 0.444, 'pos': 0.556, 'compound': 0.3612},
 {'neg': 0.787, 'neu': 0.213, 'pos': 0.0, 'compound': -0.8126},
 {'neg': 0.0, 'neu': 0.0, 'pos': 1.0, 'compound': 0.4588},
 {'neg': 0.0, 'neu': 0.707, 'pos': 0.293, 'compound': 0.4404},
 {'neg': 0.123, 'neu': 0.722, 'pos': 0.155, 'compound': 0.3704},
 {'neg': 0.034, 'neu': 0.849, 'pos': 0.117, 'compound': 0.8847},
 {'neg': 0.239, 'neu': 0.513, 'pos': 0.248, 'compound': 0.0258},
 {'neg

In [17]:
df_scores = df.from_dict(scores)
df_scores.head()

,neg,neu,pos,compound
0,0.155,0.700,0.145,-0.0516
1,0.000,1.000,0.000,0.0000
2,0.048,0.868,0.084,0.6243
3,0.000,0.830,0.170,0.8402
4,0.000,1.000,0.000,0.0000


In [18]:
df2['neg'] = df_scores['neg']
df2['neutral'] = df_scores['neu']
df2['positive'] = df_scores['pos']
df2['compound'] = df_scores['compound']
df2 = df2.drop('score', axis=1)
df2.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound
0,2701393,5l,5l,2011-06-27 14:28:01+00:00,and the glazier who fixed all the broken windo...,2701243,0,0.155,0.700,0.145,-0.0516
1,5811403,99,99,2013-06-03 04:34:08+00:00,does canada have the equivalent of h1bgreen ca...,5804452,0,0.000,1.000,0.000,0.0000
2,21623,AF,AF,2007-05-12 17:53:20+00:00,speaking of rails there are other options in t...,21611,0,0.048,0.868,0.084,0.6243
3,10159727,EA,EA,2015-09-02 15:09:34+00:00,humans and large livestock and maybe even pets...,10159396,0,0.000,0.830,0.170,0.8402
4,2988424,Iv,Iv,2011-09-12 18:53:00+00:00,i must say i reacted in the same way when i re...,2988179,0,0.000,1.000,0.000,0.0000


In [19]:
df3 = df2.sort_values(by='compound')
df3.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound
7179,10277135,DanBC,DanBC,2015-09-25 09:41:23+00:00,drug use has clear links with violence althoug...,10276081,0,0.276,0.652,0.072,-0.9996
19788,9552225,saurik,saurik,2015-05-15 17:39:30+00:00,c allows either error cases and nonerror cases...,9551196,0,0.199,0.736,0.065,-0.9982
428,10306906,Natsu,Natsu,2015-09-30 20:28:30+00:00,i know almost nothing about this case but from...,10306574,0,0.139,0.805,0.055,-0.9982
27620,6909187,omegant,omegant,2013-12-15 12:39:14+00:00,airline pilot here with experience in b737 md ...,6907771,0,0.148,0.803,0.049,-0.9979
28254,9834233,rayalez,rayalez,2015-07-05 17:39:53+00:00,pprecode gt while not appearing to assign m...,9834171,0,0.250,0.702,0.048,-0.9977


In [20]:
#df3 = df3.groupby(by='author')
df3.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound
7179,10277135,DanBC,DanBC,2015-09-25 09:41:23+00:00,drug use has clear links with violence althoug...,10276081,0,0.276,0.652,0.072,-0.9996
19788,9552225,saurik,saurik,2015-05-15 17:39:30+00:00,c allows either error cases and nonerror cases...,9551196,0,0.199,0.736,0.065,-0.9982
428,10306906,Natsu,Natsu,2015-09-30 20:28:30+00:00,i know almost nothing about this case but from...,10306574,0,0.139,0.805,0.055,-0.9982
27620,6909187,omegant,omegant,2013-12-15 12:39:14+00:00,airline pilot here with experience in b737 md ...,6907771,0,0.148,0.803,0.049,-0.9979
28254,9834233,rayalez,rayalez,2015-07-05 17:39:53+00:00,pprecode gt while not appearing to assign m...,9834171,0,0.250,0.702,0.048,-0.9977


In [21]:
#Logic wound up being poor for actually generating total scores
#Tried adding in the list of unique user names to make the math work
scores = []
users = df3["author"].unique()
for i in users:
    score = df3.loc[df3['author'] == i, 'compound'].sum()
    scores.append(score)
scores[0:5]

[2.1544000000000003, 4.012100000000001, 0.8482000000000005, -2.4314, -0.655]

In [22]:
scores_df = pd.DataFrame(df3['author'].unique())
scores_df.head()

,0
0,DanBC
1,saurik
2,Natsu
3,omegant
4,rayalez


In [23]:
scores_df['Scores'] = scores
scores_df.head()

,0,Scores
0,DanBC,2.1544
1,saurik,4.0121
2,Natsu,0.8482
3,omegant,-2.4314
4,rayalez,-0.6550


In [24]:
scores_df.sort_values(by='Scores', ascending=True)

,0,Scores
474,jlgreco,-5.6119
32,wnight,-5.4195
826,tsotha,-4.6735
404,GHFigs,-3.2631
232,tedks,-3.2189
...,...,...
73,rbanffy,24.5337
214,rdl,24.5744
52,patio11,28.2934
87,jerf,28.6256


In [25]:
column_dict = {0:'author'}
scores_df = scores_df.rename(column_dict, axis=1)

In [26]:
df3.loc[df3['author'] == 'jlgreco', 'compound'].sum()

-5.6119

In [33]:
df4 = df3.merge(scores_df)
#df4 = df4.drop(['ranking', 'Total_Score'], axis=1)
df4.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound,Scores
0,10277135,DanBC,DanBC,2015-09-25 09:41:23+00:00,drug use has clear links with violence althoug...,10276081,0,0.276,0.652,0.072,-0.9996,2.1544
1,8013915,DanBC,DanBC,2014-07-10 06:37:37+00:00,you still havent said whether you are respondi...,8013637,0,0.196,0.747,0.057,-0.9963,2.1544
2,4849637,DanBC,DanBC,2012-11-29 19:08:01+00:00,trading in snuff films is not legal in all reg...,4848813,0,0.335,0.646,0.019,-0.9938,2.1544
3,5952124,DanBC,DanBC,2013-06-27 13:27:01+00:00,gt they got locked up and then sectioned indef...,5952052,0,0.170,0.772,0.059,-0.9779,2.1544
4,3728783,DanBC,DanBC,2012-03-20 11:47:39+00:00,yes poor people migrate to where they think t...,3728720,0,0.243,0.706,0.051,-0.9623,2.1544


In [34]:
column_dict = {"Scores":"Total_Score"}
df4 = df4.rename(column_dict, axis=1)

In [35]:
df4.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound,Total_Score
0,10277135,DanBC,DanBC,2015-09-25 09:41:23+00:00,drug use has clear links with violence althoug...,10276081,0,0.276,0.652,0.072,-0.9996,2.1544
1,8013915,DanBC,DanBC,2014-07-10 06:37:37+00:00,you still havent said whether you are respondi...,8013637,0,0.196,0.747,0.057,-0.9963,2.1544
2,4849637,DanBC,DanBC,2012-11-29 19:08:01+00:00,trading in snuff films is not legal in all reg...,4848813,0,0.335,0.646,0.019,-0.9938,2.1544
3,5952124,DanBC,DanBC,2013-06-27 13:27:01+00:00,gt they got locked up and then sectioned indef...,5952052,0,0.170,0.772,0.059,-0.9779,2.1544
4,3728783,DanBC,DanBC,2012-03-20 11:47:39+00:00,yes poor people migrate to where they think t...,3728720,0,0.243,0.706,0.051,-0.9623,2.1544


In [36]:
df4.sort_values(by=["Total_Score", "compound"])

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound,Total_Score
6106,5046993,jlgreco,jlgreco,2013-01-12 08:26:59+00:00,we have no problem directing our attention at ...,5046956,0,0.342,0.658,0.000,-0.9109,-5.6119
6107,5230849,jlgreco,jlgreco,2013-02-16 08:07:35+00:00,are you familiar with what caning as performed...,5230787,0,0.161,0.766,0.072,-0.9052,-5.6119
6108,6419797,jlgreco,jlgreco,2013-09-20 18:57:52+00:00,a fizzle of the relatively small fission prima...,6419640,0,0.121,0.867,0.011,-0.9032,-5.6119
6109,6747953,jlgreco,jlgreco,2013-11-17 07:11:56+00:00,that possibility is where this section applies...,6746255,0,0.221,0.742,0.037,-0.8720,-5.6119
6110,5188700,jlgreco,jlgreco,2013-02-08 16:43:21+00:00,petitioning to have a government official fire...,5188560,0,0.190,0.759,0.051,-0.8643,-5.6119
...,...,...,...,...,...,...,...,...,...,...,...,...
8793,5509924,eru,eru,2013-04-08 04:03:16+00:00,you can also ask a trusted friend just ask th...,5497159,0,0.000,0.593,0.407,0.8625,30.7788
8794,2506281,eru,eru,2011-05-02 14:02:06+00:00,standard deviation is the square root of varia...,2506029,0,0.000,0.873,0.127,0.9050,30.7788
8795,1200096,eru,eru,2010-03-18 00:24:44+00:00,oh theres something to be said for having an o...,1200064,0,0.000,0.731,0.269,0.9056,30.7788
8796,915326,eru,eru,2009-11-01 18:01:14+00:00,depends on what you mean by every thing else ...,915218,0,0.000,0.722,0.278,0.9325,30.7788


In [37]:
df4.head()

,id,by,author,time_ts,text,parent,ranking,neg,neutral,positive,compound,Total_Score
0,10277135,DanBC,DanBC,2015-09-25 09:41:23+00:00,drug use has clear links with violence althoug...,10276081,0,0.276,0.652,0.072,-0.9996,2.1544
1,8013915,DanBC,DanBC,2014-07-10 06:37:37+00:00,you still havent said whether you are respondi...,8013637,0,0.196,0.747,0.057,-0.9963,2.1544
2,4849637,DanBC,DanBC,2012-11-29 19:08:01+00:00,trading in snuff films is not legal in all reg...,4848813,0,0.335,0.646,0.019,-0.9938,2.1544
3,5952124,DanBC,DanBC,2013-06-27 13:27:01+00:00,gt they got locked up and then sectioned indef...,5952052,0,0.170,0.772,0.059,-0.9779,2.1544
4,3728783,DanBC,DanBC,2012-03-20 11:47:39+00:00,yes poor people migrate to where they think t...,3728720,0,0.243,0.706,0.051,-0.9623,2.1544


2.1544000000000003